In [ ]:
from flask import Flask, request, render_template, send_file, redirect, url_for, flash
import threading
import shelve
from joblib import dump, load
from datetime import datetime
import xlsxwriter
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from flask import template_rendered
from contextlib import contextmanager

app = Flask(__name__)
app.secret_key = b'_5#y2L"F4Q8z\n\xec]/'
# Use a service account
cred = credentials.Certificate('static/serviceAccount.json')
firebase_admin.initialize_app(cred)

db = firestore.client()


# Create an Event for notifying main thread.
callback_done = threading.Event()

tables=[u"aircraft_reg_no",u"aircraft_type",u"customer_program",u"engine_type",u"inspected_by",u"location",u"trade"]
table_data_list=[]
table_data={}

def get_data(collection_ref):
    temp_list=[]
    docs = db.collection(collection_ref).get()
    for doc in docs:
        temp = doc.to_dict()
        temp['id'] = doc.id
        temp_list.append(temp)
    return temp_list
     
    
def find_array_index_by_name(array,name):
    for i in range(len(array)):
        if array[i]["name"] == name:
            return i
    return False

# Create a callback on_snapshot function to capture changes
def on_snapshot(doc_snapshot, changes, read_time):
    for doc in doc_snapshot:
        print(f'Received document snapshot: {doc.to_dict()}')
    callback_done.set()
    
    
@app.route('/')
def index():
    return render_template('index.html')



@app.route('/crud')
def crud():
    return render_template('addData.html',table=table_data, table_names=tables)


@app.route('/selectTable', methods=['POST'])
def select_table():
    collection_id = request.form["collection_id"]
    res = get_data(collection_id)
    table_data["name"] = collection_id
    table_data["data"] = res
    return render_template('addData.html',table=table_data,table_names=tables)


@app.route('/add/<coll_id>', methods=['POST'])
def add(coll_id):
    coll_id=str(coll_id)
    db.collection(coll_id).add({coll_id:str(request.form['name'])})
    res = get_data(coll_id)
    table_data["data"]=res
    flash("data added")
    return render_template('addData.html',table=table_data,table_names=tables)



@app.route('/startEdit/<i>', methods=['GET'])
def startEdit(i):
    i=int(i)
    print(table_data)
    for data in table_data["data"]:
        data["editing"] = False
    table_data["data"][i]["editing"]=True
    return render_template('addData.html',table=table_data,table_names=tables)


@app.route('/edit/<coll_id>/<docId>', methods=['POST'])
def edit(coll_id,docId):
    coll_id=str(coll_id)
    db.collection(coll_id).document(str(docId)).update({coll_id: str(request.form['name'])})
    res = get_data(coll_id)
    table_data["data"]=res
    for data in table_data["data"]:
        data["editing"] = False
    flash("data Updated")
    return render_template('addData.html',table=table_data,table_names=tables)



@app.route('/delete/<coll_id>/<docId>', methods=['GET'])
def delete(coll_id,docId):
    coll_id=str(coll_id)
    db.collection(coll_id).document(str(docId)).delete()
    res = get_data(coll_id)
    table_data["data"]=res
    flash("data deleted")
    return render_template('addData.html',table=table_data,table_names=tables)

if __name__ == "__main__":
    app.run()
